In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
from glob2 import glob
import osmnx as ox

## Load data using OSMnx 
https://osmnx.readthedocs.io/en/stable/osmnx.html#module-osmnx.geometries

In [2]:
green=ox.geometries.geometries_from_bbox(51.691874,51.28676,0.334016,-0.510375,
                                         {'leisure':'park', 'landuse':'forest'})

In [3]:
#keep only polygons

#exclude Point
green=green[green['geometry'].geom_type.ne('Point')]
#exclude LineString
green=green[green['geometry'].geom_type.ne('LineString')]

In [4]:
green.head()

,unique_id,osmid,element_type,highway,geometry,created_by,source,barrier,name,access,...,multipolygon,contact:email,start_date:note,disused,previous_id,access_land,leisure_1,architect,previously,source:geometry
11,way/3184264,3184264,way,NaN,"POLYGON ((-0.33801 51.29715, -0.33716 51.29791...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,way/3385169,3385169,way,NaN,"POLYGON ((-0.41102 51.44907, -0.41079 51.44955...",NaN,NaN,NaN,Glebelands Playing Fields Park,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,way/3401837,3401837,way,NaN,"POLYGON ((-0.38753 51.42046, -0.38732 51.42062...",NaN,NaN,NaN,Hatherop Park,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,way/3430579,3430579,way,NaN,"POLYGON ((-0.36641 51.42020, -0.36736 51.41948...",NaN,NaN,NaN,Carlisle Park,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,way/3541791,3541791,way,NaN,"POLYGON ((-0.34399 51.40805, -0.34262 51.40746...",NaN,NaN,NaN,Hampton Court Green,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
green.shape

(5446, 171)

In [6]:
green.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

## Exclude the overlapping geometries 

In [7]:
geom = green.geometry.unary_union

In [8]:
green_unique = gpd.GeoDataFrame(geometry=[geom],crs='epsg:4326')

In [9]:
green_unique.head()

,geometry
0,"MULTIPOLYGON (((-0.24209 51.45606, -0.24205 51..."


In [10]:
green_unique.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [11]:
green_unique = green_unique.explode().reset_index(drop=True)

In [12]:
green_unique.head()

,geometry
0,"POLYGON ((-0.24209 51.45606, -0.24205 51.45651..."
1,"POLYGON ((-0.03133 51.43362, -0.03034 51.43356..."
2,"POLYGON ((0.06302 51.43869, 0.06307 51.43878, ..."
3,"POLYGON ((-0.09444 51.28794, -0.09155 51.28767..."
4,"POLYGON ((0.20140 51.28944, 0.20134 51.28899, ..."


In [13]:
green_unique.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [14]:
green_unique.shape

(4838, 1)

## Load observations

In [15]:
data_folder = '../data/observations_final'
london_df = pd.concat([pd.read_csv(f).assign(challenge=f.replace('.csv','')) for f in glob(data_folder+'/CNC_London_*.csv')])

In [16]:
london_df.shape

(13750, 39)

In [17]:
london_df.head()

,id,observed_on_string,time_observed_at,created_time_zone,created_at,updated_at,description,user_id,user_login,quality_grade,...,iconic_taxon_name,taxon_rank,taxon_parent_id,taxon_native,taxon_endemic,taxon_threatened,taxon_search_rank,taxon_observations,identifications,challenge
0,11479007,2018-04-27 7:52:34 am BST,2018-04-27T07:52:34+00:00,Etc/UTC,2018-04-27T07:06:17+00:00,2018-04-27T09:26:56+00:00,NaN,159021,muki,research,...,Plantae,species,204212,True,False,False,27624,27624,"[{'user_id': 353381, 'category': 'improving', ...",../data/observations_final\CNC_London_2018
1,11480212,2018-04-27 8:39:41 am BST,2018-04-27T08:39:41+01:00,Europe/London,2018-04-27T09:02:52+01:00,2019-01-29T21:16:34+00:00,NaN,664459,lucyrobinsonnhm,research,...,Aves,variety,3017,False,False,False,43329,43329,"[{'user_id': 29729, 'category': 'supporting', ...",../data/observations_final\CNC_London_2018
2,11480902,Fri Apr 27 2018 08:34:31 GMT+0100 (GMT+1),2018-04-27T08:34:31+02:00,Europe/Amsterdam,2018-04-27T10:42:01+02:00,2018-04-27T13:01:49+02:00,NaN,908315,tess26,casual,...,Plantae,species,56095,False,False,False,2813,2813,"[{'user_id': 908315, 'category': 'leading', 'd...",../data/observations_final\CNC_London_2018
3,11481097,Fri Apr 27 2018 09:54:35 GMT+0100 (GMT+1),2018-04-27T09:54:35+02:00,Europe/Amsterdam,2018-04-27T10:54:45+02:00,2018-04-27T22:23:41+02:00,NaN,796473,bryonycross,research,...,Plantae,species,156638,False,False,False,4639,4639,"[{'user_id': 160, 'category': 'improving', 'di...",../data/observations_final\CNC_London_2018
4,11482247,2018-04-27 8:42:21 am BST,2018-04-27T08:42:21+01:00,Europe/London,2018-04-27T11:09:55+01:00,2018-04-27T16:37:31+01:00,NaN,664459,lucyrobinsonnhm,research,...,Plantae,hybrid,49664,False,False,False,5866,5866,"[{'user_id': 160, 'category': 'supporting', 'd...",../data/observations_final\CNC_London_2018


In [18]:
london_df['latitude'], london_df['longitude']

(0       51.550636
 1       51.495277
 2       51.472981
 3       51.508076
 4       51.496113
           ...    
 5798    51.481135
 5799    51.511718
 5800    51.360768
 5801    51.550848
 5802    51.464695
 Name: latitude, Length: 13750, dtype: float64, 0      -0.228487
 1      -0.173611
 2      -0.169102
 3      -0.161516
 4      -0.173889
           ...   
 5798    0.112511
 5799   -0.030521
 5800   -0.160632
 5801   -0.135313
 5802   -0.226758
 Name: longitude, Length: 13750, dtype: float64)

In [19]:
#create the points inverting the coordinates
geometry_invert = [Point(xy) for xy in zip(london_df.longitude, london_df.latitude)]

In [20]:
#crete a geodataframe with the observations
observations_gdf = gpd.GeoDataFrame(london_df, crs='epsg:4326', geometry=geometry_invert)

In [21]:
observations_gdf.shape

(13750, 40)

In [22]:
observations_gdf.head()

,id,observed_on_string,time_observed_at,created_time_zone,created_at,updated_at,description,user_id,user_login,quality_grade,...,taxon_rank,taxon_parent_id,taxon_native,taxon_endemic,taxon_threatened,taxon_search_rank,taxon_observations,identifications,challenge,geometry
0,11479007,2018-04-27 7:52:34 am BST,2018-04-27T07:52:34+00:00,Etc/UTC,2018-04-27T07:06:17+00:00,2018-04-27T09:26:56+00:00,NaN,159021,muki,research,...,species,204212,True,False,False,27624,27624,"[{'user_id': 353381, 'category': 'improving', ...",../data/observations_final\CNC_London_2018,POINT (-0.22849 51.55064)
1,11480212,2018-04-27 8:39:41 am BST,2018-04-27T08:39:41+01:00,Europe/London,2018-04-27T09:02:52+01:00,2019-01-29T21:16:34+00:00,NaN,664459,lucyrobinsonnhm,research,...,variety,3017,False,False,False,43329,43329,"[{'user_id': 29729, 'category': 'supporting', ...",../data/observations_final\CNC_London_2018,POINT (-0.17361 51.49528)
2,11480902,Fri Apr 27 2018 08:34:31 GMT+0100 (GMT+1),2018-04-27T08:34:31+02:00,Europe/Amsterdam,2018-04-27T10:42:01+02:00,2018-04-27T13:01:49+02:00,NaN,908315,tess26,casual,...,species,56095,False,False,False,2813,2813,"[{'user_id': 908315, 'category': 'leading', 'd...",../data/observations_final\CNC_London_2018,POINT (-0.16910 51.47298)
3,11481097,Fri Apr 27 2018 09:54:35 GMT+0100 (GMT+1),2018-04-27T09:54:35+02:00,Europe/Amsterdam,2018-04-27T10:54:45+02:00,2018-04-27T22:23:41+02:00,NaN,796473,bryonycross,research,...,species,156638,False,False,False,4639,4639,"[{'user_id': 160, 'category': 'improving', 'di...",../data/observations_final\CNC_London_2018,POINT (-0.16152 51.50808)
4,11482247,2018-04-27 8:42:21 am BST,2018-04-27T08:42:21+01:00,Europe/London,2018-04-27T11:09:55+01:00,2018-04-27T16:37:31+01:00,NaN,664459,lucyrobinsonnhm,research,...,hybrid,49664,False,False,False,5866,5866,"[{'user_id': 160, 'category': 'supporting', 'd...",../data/observations_final\CNC_London_2018,POINT (-0.17389 51.49611)


In [23]:
observations_gdf['longitude'].isna().sum()

0

## Spatial join

In [24]:
# spatial join observations and observations.
points_polys = gpd.sjoin(observations_gdf, green_unique, how='inner')

In [25]:
points_polys.head()

,id,observed_on_string,time_observed_at,created_time_zone,created_at,updated_at,description,user_id,user_login,quality_grade,...,taxon_parent_id,taxon_native,taxon_endemic,taxon_threatened,taxon_search_rank,taxon_observations,identifications,challenge,geometry,index_right
3,11481097,Fri Apr 27 2018 09:54:35 GMT+0100 (GMT+1),2018-04-27T09:54:35+02:00,Europe/Amsterdam,2018-04-27T10:54:45+02:00,2018-04-27T22:23:41+02:00,NaN,796473,bryonycross,research,...,156638,False,False,False,4639,4639,"[{'user_id': 160, 'category': 'improving', 'di...",../data/observations_final\CNC_London_2018,POINT (-0.16152 51.50808),2551
5,11482706,Fri Apr 27 2018 11:40:06 GMT+0100 (GMT+1),2018-04-27T11:40:06+02:00,Europe/Amsterdam,2018-04-27T12:40:31+02:00,2018-04-27T19:18:08+02:00,NaN,796473,bryonycross,casual,...,861033,False,False,False,6135,6135,"[{'user_id': 796473, 'category': 'leading', 'd...",../data/observations_final\CNC_London_2018,POINT (-0.16758 51.51051),2551
6,11483083,Fri Apr 27 2018 12:01:15 GMT+0100 (GMT+1),2018-04-27T12:01:15+02:00,Europe/Amsterdam,2018-04-27T13:01:48+02:00,2018-04-27T19:18:09+02:00,NaN,796473,bryonycross,needs_id,...,63089,False,False,False,42225,42225,"[{'user_id': 796473, 'category': 'leading', 'd...",../data/observations_final\CNC_London_2018,POINT (-0.16793 51.51077),2551
10,11484564,Fri Apr 27 2018 12:58:17 GMT+0100 (GMT+1),2018-04-27T12:58:17+02:00,Europe/Amsterdam,2018-04-27T13:58:34+02:00,2018-04-27T19:18:10+02:00,"Large tree, medium sized leaves that are mostl...",796473,bryonycross,research,...,87634,False,False,False,28819,28819,"[{'user_id': 796473, 'category': 'improving', ...",../data/observations_final\CNC_London_2018,POINT (-0.16176 51.50792),2551
11,11484587,Fri Apr 27 2018 12:58:55 GMT+0100 (GMT+1),2018-04-27T12:58:55+02:00,Europe/Amsterdam,2018-04-27T13:59:13+02:00,2018-04-27T19:18:11+02:00,"Large tree, medium sized leaves that are mostl...",796473,bryonycross,needs_id,...,549548,False,False,False,4080,4080,"[{'user_id': 796473, 'category': 'leading', 'd...",../data/observations_final\CNC_London_2018,POINT (-0.16168 51.50783),2551


In [26]:
points_polys.shape

(5828, 41)

In [27]:
points_polys.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

## Set flag for greenspaces

In [28]:
ids_with_greenspace = points_polys.id.to_list()

In [29]:
new_london = london_df.set_index('id')
new_london.head()

,observed_on_string,time_observed_at,created_time_zone,created_at,updated_at,description,user_id,user_login,quality_grade,reviewed_by,...,taxon_rank,taxon_parent_id,taxon_native,taxon_endemic,taxon_threatened,taxon_search_rank,taxon_observations,identifications,challenge,geometry
id,,,,,,,,,,,,,,,,,,,,,
11479007,2018-04-27 7:52:34 am BST,2018-04-27T07:52:34+00:00,Etc/UTC,2018-04-27T07:06:17+00:00,2018-04-27T09:26:56+00:00,NaN,159021,muki,research,"[160, 353381, 599476, 664459, 743476]",...,species,204212,True,False,False,27624,27624,"[{'user_id': 353381, 'category': 'improving', ...",../data/observations_final\CNC_London_2018,POINT (-0.22849 51.55064)
11480212,2018-04-27 8:39:41 am BST,2018-04-27T08:39:41+01:00,Europe/London,2018-04-27T09:02:52+01:00,2019-01-29T21:16:34+00:00,NaN,664459,lucyrobinsonnhm,research,"[29729, 42615, 599476, 664459, 790453, 1082970]",...,variety,3017,False,False,False,43329,43329,"[{'user_id': 29729, 'category': 'supporting', ...",../data/observations_final\CNC_London_2018,POINT (-0.17361 51.49528)
11480902,Fri Apr 27 2018 08:34:31 GMT+0100 (GMT+1),2018-04-27T08:34:31+02:00,Europe/Amsterdam,2018-04-27T10:42:01+02:00,2018-04-27T13:01:49+02:00,NaN,908315,tess26,casual,[908315],...,species,56095,False,False,False,2813,2813,"[{'user_id': 908315, 'category': 'leading', 'd...",../data/observations_final\CNC_London_2018,POINT (-0.16910 51.47298)
11481097,Fri Apr 27 2018 09:54:35 GMT+0100 (GMT+1),2018-04-27T09:54:35+02:00,Europe/Amsterdam,2018-04-27T10:54:45+02:00,2018-04-27T22:23:41+02:00,NaN,796473,bryonycross,research,"[160, 353381, 599476, 796473]",...,species,156638,False,False,False,4639,4639,"[{'user_id': 160, 'category': 'improving', 'di...",../data/observations_final\CNC_London_2018,POINT (-0.16152 51.50808)
11482247,2018-04-27 8:42:21 am BST,2018-04-27T08:42:21+01:00,Europe/London,2018-04-27T11:09:55+01:00,2018-04-27T16:37:31+01:00,NaN,664459,lucyrobinsonnhm,research,"[160, 353381, 599476, 664459]",...,hybrid,49664,False,False,False,5866,5866,"[{'user_id': 160, 'category': 'supporting', 'd...",../data/observations_final\CNC_London_2018,POINT (-0.17389 51.49611)


In [30]:
new_london['greenspace_flag'] = '0'
new_london.loc[ids_with_greenspace,'greenspace_flag'] = '1'

In [31]:
new_london.greenspace_flag.value_counts()

0    7922
1    5828
Name: greenspace_flag, dtype: int64

In [32]:
new_london.head()

,observed_on_string,time_observed_at,created_time_zone,created_at,updated_at,description,user_id,user_login,quality_grade,reviewed_by,...,taxon_parent_id,taxon_native,taxon_endemic,taxon_threatened,taxon_search_rank,taxon_observations,identifications,challenge,geometry,greenspace_flag
id,,,,,,,,,,,,,,,,,,,,,
11479007,2018-04-27 7:52:34 am BST,2018-04-27T07:52:34+00:00,Etc/UTC,2018-04-27T07:06:17+00:00,2018-04-27T09:26:56+00:00,NaN,159021,muki,research,"[160, 353381, 599476, 664459, 743476]",...,204212,True,False,False,27624,27624,"[{'user_id': 353381, 'category': 'improving', ...",../data/observations_final\CNC_London_2018,POINT (-0.22849 51.55064),0
11480212,2018-04-27 8:39:41 am BST,2018-04-27T08:39:41+01:00,Europe/London,2018-04-27T09:02:52+01:00,2019-01-29T21:16:34+00:00,NaN,664459,lucyrobinsonnhm,research,"[29729, 42615, 599476, 664459, 790453, 1082970]",...,3017,False,False,False,43329,43329,"[{'user_id': 29729, 'category': 'supporting', ...",../data/observations_final\CNC_London_2018,POINT (-0.17361 51.49528),0
11480902,Fri Apr 27 2018 08:34:31 GMT+0100 (GMT+1),2018-04-27T08:34:31+02:00,Europe/Amsterdam,2018-04-27T10:42:01+02:00,2018-04-27T13:01:49+02:00,NaN,908315,tess26,casual,[908315],...,56095,False,False,False,2813,2813,"[{'user_id': 908315, 'category': 'leading', 'd...",../data/observations_final\CNC_London_2018,POINT (-0.16910 51.47298),0
11481097,Fri Apr 27 2018 09:54:35 GMT+0100 (GMT+1),2018-04-27T09:54:35+02:00,Europe/Amsterdam,2018-04-27T10:54:45+02:00,2018-04-27T22:23:41+02:00,NaN,796473,bryonycross,research,"[160, 353381, 599476, 796473]",...,156638,False,False,False,4639,4639,"[{'user_id': 160, 'category': 'improving', 'di...",../data/observations_final\CNC_London_2018,POINT (-0.16152 51.50808),1
11482247,2018-04-27 8:42:21 am BST,2018-04-27T08:42:21+01:00,Europe/London,2018-04-27T11:09:55+01:00,2018-04-27T16:37:31+01:00,NaN,664459,lucyrobinsonnhm,research,"[160, 353381, 599476, 664459]",...,49664,False,False,False,5866,5866,"[{'user_id': 160, 'category': 'supporting', 'd...",../data/observations_final\CNC_London_2018,POINT (-0.17389 51.49611),0


In [33]:
#export as csv
new_london.to_csv('../data/outputs/london_data_with_greenspace_flag_v2.csv')

In [34]:
#compare with Florence's csv export
new_london_v0=pd.read_csv('../data/outputs/london_data_with_greenspace_flag_v1.csv')

In [35]:
new_london_v0.shape, new_london.shape

((13730, 38), (13750, 40))

In [36]:
#no columns E and N in the new df
new_london_v0.columns, new_london.columns

(Index(['id', 'observed_on_string', 'observed_on', 'time_observed_at',
        'time_zone', 'user_id', 'user_login', 'created_at', 'updated_at',
        'quality_grade', 'license', 'url', 'image_url', 'sound_url', 'tag_list',
        'description', 'num_identification_agreements',
        'num_identification_disagreements', 'captive_cultivated',
        'oauth_application_id', 'place_guess', 'latitude', 'longitude',
        'positional_accuracy', 'geoprivacy', 'taxon_geoprivacy',
        'coordinates_obscured', 'positioning_method', 'positioning_device',
        'species_guess', 'scientific_name', 'common_name', 'iconic_taxon_name',
        'taxon_id', 'challenge', 'E', 'N', 'greenspace_flag'],
       dtype='object'),
 Index(['observed_on_string', 'time_observed_at', 'created_time_zone',
        'created_at', 'updated_at', 'description', 'user_id', 'user_login',
        'quality_grade', 'reviewed_by', 'faves_count',
        'num_identification_agreements', 'num_identification_disagreem

In [37]:
new_london_v0.greenspace_flag.value_counts()

0    8235
1    5495
Name: greenspace_flag, dtype: int64

In [38]:
new_london.greenspace_flag.value_counts()

0    7922
1    5828
Name: greenspace_flag, dtype: int64

In [39]:
5828-5495

333

#### 333 more observations (n=13730) with greenspace_flag = 1 using osmnx to get OSM data and observations downloaded with the API if compared with analysis using the dump files